We will use the [API](https://dadosabertos.camara.leg.br/swagger/api.html) in order to retrieve information about the deputies and theirs speechs. Then, we can merge them all into a single dataset.

In [0]:
import requests
import pandas as pd

import sys

## Retrieving the speeches

First, we must retrieve the id information about all the deputies

In [2]:
response = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados/')

json_data = response.json()

# Print out the first 3 deputies
json_data['dados'][:3]

[{'id': 204554,
  'idLegislatura': 56,
  'nome': 'ABÍLIO SANTANA',
  'siglaPartido': 'PL',
  'siglaUf': 'BA',
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204554',
  'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
  'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204554.jpg'},
 {'id': 204521,
  'idLegislatura': 56,
  'nome': 'ABOU ANNI',
  'siglaPartido': 'PSL',
  'siglaUf': 'SP',
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204521',
  'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36837',
  'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204521.jpg'},
 {'id': 204379,
  'idLegislatura': 56,
  'nome': 'ACÁCIO FAVACHO',
  'siglaPartido': 'PROS',
  'siglaUf': 'AP',
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204379',
  'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36763',
  'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204379.j

In [0]:
deputies = []

for data in json_data['dados']:
  deputies.append([data['id'], data['siglaPartido'], data['siglaUf']])

---
Now that we have the ids from each deputy, we can make a query that returns all speeches given between 1st January and 30 June this year (2019).

In [4]:
speeches = []

total = len(deputies)
# Create a dictionary that contains
# id : speech1, speech2
for i, dep in enumerate(deputies):
  sys.stdout.write("\rGetting %i of %i" % (i, total))
  sys.stdout.flush()
  
  # Get all speechs given by the current deputy
  query = 'https://dadosabertos.camara.leg.br/api/v2/deputados/'+str(dep[0])+\
          '/discursos?dataInicio=2019-01-01&dataFim=2019-06-11&ordenarPor=dataHoraInicio&ordem=ASC'
  response = requests.get(query)
  speech_json = response.json()
  
  # Add each speech to the list
  speeches += [[ dep[0], speech['transcricao'], dep[1], dep[2] ] for speech in speech_json['dados']]
  
sys.stdout.write("\rExtraction complete! %i speeches retrieved." % len(speeches))

Extraction complete! 3938 speeches retrieved.

In [5]:
# Check out the first 5 rows
speeches[:5]

[[204554,
  'O SR. ABÍLIO SANTANA (PHS - BA. Pela ordem. Sem revisão do orador.) - Sr. Presidente, douta Mesa, senhoras e senhores, trago uma saudação de paz.\r\nEu sou oriundo da Bahia, o quarto Estado brasileiro em população e o sexto maior Estado, geograficamente falando, do nosso querido Brasil. Eu tenho o privilégio de proferir com muita segurança que nasci no Estado onde tudo começou, onde as caravelas ancoraram, onde Cabral bradou: "Terra à vista!" \r\nDos 417 Municípios do meu querido Estado da Bahia, 410 acreditaram em mim e me agraciaram com seus votos. Quando me enviaram para Brasília, Capital da nossa Nação, fizeram-no acreditando no que eu proferi durante a campanha: eu serei radicalmente contra todos os que se levantarem contra a ética e a família tradicional. Levantar-me-ei contra quem é a favor da chamada ideologia de gênero, que, pela minha ótica, até que me provem o contrário, tem sido uma catástrofe para nossa querida Nação.\r\nAgradeço ao Deus dos céus por ter me ag

## Create the dataset

With the id, speech, party and state, we are able to create a dataframe containing all this information. Thus, we can in the future aggregate all speeches by speaker, party or state, and make some other processings.

In [0]:
dataset = pd.DataFrame(speeches, columns=['id', 'speech', 'party', 'state'])

In [54]:
size = dataset.shape[0]

size

3938

Now, let's take a look at the result

In [55]:
dataset.head()

,id,speech,party,state
0,204554,O SR. ABÍLIO SANTANA (PHS - BA. Pela ordem. Se...,PL,BA
1,204554,O SR. ABÍLIO SANTANA (PHS - BA. Pela ordem. Se...,PL,BA
2,204521,O SR. ABOU ANNI (PSL - SP. Sem revisão do orad...,PSL,SP
3,204521,O SR. ABOU ANNI (PSL - SP. Sem revisão do orad...,PSL,SP
4,204560,O SR. ADOLFO VIANA (Bloco/PSDB - BA. Sem revis...,PSDB,BA


In [56]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3938 entries, 0 to 3937
Data columns (total 4 columns):
id        3938 non-null int64
speech    3938 non-null object
party     3938 non-null object
state     3938 non-null object
dtypes: int64(1), object(3)
memory usage: 123.1+ KB


Awesome! We have no missing data!

In [57]:
dataset[['party', 'state']].describe()

,party,state
count,3938,3938
unique,26,27
top,PT,SP
freq,654,551


Interesting... The people from PT party are the ones who gave more speeches this year so far. When analysing the state, deputies from SP.

### Processing

By quickly observing the speeches, it's visible that they some of them begin in that way:

```
O SR. ABOU ANNI (PSL - SP. Sem revisão do orador.) - 
```
or,

```
A SRA. NATÁLIA BONAVIDES (PT - RN. Pela ordem. Sem revisão da oradora.) - 
```

Besides, there  are some speeches that starts with:

```
DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPUTADO BILAC PINTO.\r\n\r\n
```

In order to remove this repetitions, we may remove this introdutory text. We divide this process in two parts: one for each type of intro. Then, we mix the data again.


We start by managing the first pattern.

In [0]:
# Create a copy
dataset1 = dataset.copy()

In [0]:
# We split the data using the signal " - " and retrieve the last string
# obtained in that split
dataset['speech'] = dataset.speech.str.split(" - ", n=2, expand=True)[2]

In [104]:
dataset.loc[0].speech

'Sr. Presidente, douta Mesa, senhoras e senhores, trago uma saudação de paz.\r\nEu sou oriundo da Bahia, o quarto Estado brasileiro em população e o sexto maior Estado, geograficamente falando, do nosso querido Brasil. Eu tenho o privilégio de proferir com muita segurança que nasci no Estado onde tudo começou, onde as caravelas ancoraram, onde Cabral bradou: "Terra à vista!" \r\nDos 417 Municípios do meu querido Estado da Bahia, 410 acreditaram em mim e me agraciaram com seus votos. Quando me enviaram para Brasília, Capital da nossa Nação, fizeram-no acreditando no que eu proferi durante a campanha: eu serei radicalmente contra todos os que se levantarem contra a ética e a família tradicional. Levantar-me-ei contra quem é a favor da chamada ideologia de gênero, que, pela minha ótica, até que me provem o contrário, tem sido uma catástrofe para nossa querida Nação.\r\nAgradeço ao Deus dos céus por ter me agraciado ao me enviar como um dos representantes da minha querida Bahia e muito esp

In [92]:
dataset.head()

,id,speech,party,state
0,204554,"Sr. Presidente, douta Mesa, senhoras e senhore...",PL,BA
1,204554,"Sr. Presidente, douta Mesa, senhoras e senhore...",PL,BA
2,204521,"Obrigado, Sr. Presidente.\r\nNesta oportunidad...",PSL,SP
3,204521,"Obrigado, Sr. Presidente.\r\nQuero cumprimenta...",PSL,SP
4,204560,"Sra. Presidente, Sras. e Srs. Deputados, peço ...",PSDB,BA


As expected, this approach does not work on the second pattern, giving us some empty speeches, as the one below

In [0]:
dataset.loc[506].speech

In [95]:
null_speeches = dataset.loc[dataset.speech.isna()]

print( "There are %i null speeches" % len(null_speeches))

There are 169 null speeches


Thus, we remove this empty speeches and follow a second a approach on the copied dataset.

In [0]:
dataset.dropna(inplace=True)

# Assert that the number of rows decreased
assert(dataset.shape[0] < size)

Now, it's time to process the 169 speeches that were not covered by the previous approach.

In [0]:
dataset1 = dataset1.loc[null_speeches.index]

assert(dataset1.shape[0] == 169)

In [0]:
# We split the data using the pattern "\r\n\r\n" and retrieve the last string
# obtained in that split
dataset1['speech'] = dataset1.speech.str.split("\r\n\r\n", n=1, expand=True)[1]

Just assuring that all speeches were retrieved and taking a look at the result.

In [106]:
assert(dataset.speech.isna().sum() == 0)

dataset1.head()

,id,speech,party,state
192,74057,"Sr. Presidente, Sras. e Srs. Deputados, na noi...",PCdoB,BA
254,204423,"Sr. Presidente, Sras. Deputadas e Srs. Deputad...",PSC,PE
255,204423,"Sr. Presidente, Sras. e Srs. Deputados, é com ...",PSC,PE
292,160553,"Sr. Presidente, Sras. e Srs. Deputados, gostar...",PSD,BA
506,141513,"Sr. Presidente, Sras. e Srs. Deputados, aprove...",DEM,MG


After this, we can merge the data together again.

In [0]:
dataset = dataset.append(dataset1)

# Assure that all the rows are there
assert(dataset.shape[0] == size)

Finally, we can save the dataset.

In [0]:
dataset.to_csv('speeches.csv')